In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings                                  # `do not disturbe` mode
warnings.filterwarnings('ignore')

import numpy as np                               # vectors and matrices
import pandas as pd                              # tables and data manipulations
import matplotlib.pyplot as plt                  # plots
import seaborn as sns                            # more plots

from dateutil.relativedelta import relativedelta # working with dates with style
from scipy.optimize import minimize              # for function minimization

import statsmodels.formula.api as smf            # statistics and econometrics
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

from itertools import product                    # some useful functions
from tqdm import tqdm_notebook



%matplotlib inline

In [ ]:
# Import
train = pd.read_csv("../input/store-sales-time-series-forecasting/train.csv")
test = pd.read_csv("../input/store-sales-time-series-forecasting/test.csv")
stores = pd.read_csv("../input/store-sales-time-series-forecasting/stores.csv")
#sub = pd.read_csv("../input/store-sales-time-series-forecasting/sample_submission.csv")   
transactions = pd.read_csv("../input/store-sales-time-series-forecasting/transactions.csv").sort_values(["store_nbr", "date"])


# Datetime
train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)
transactions["date"] = pd.to_datetime(transactions.date)

# Data types
train.onpromotion = train.onpromotion.astype("float16")
train.sales = train.sales.astype("float32")
stores.cluster = stores.cluster.astype("int8")

train.head()

In [ ]:
# Veriyi yükleyin
df = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')

# 'family' sütununda sadece 'BOOKS' değerini içeren satırları seçin
store_number = 11  # İstediğiniz mağaza numarasını buraya girin
books_df = df[(df['family'] == 'GROCERY I') & (df['store_nbr'] == store_number)]


# Seçilen veriyi kontrol edin
print(books_df.head())

# Yeni veri setini bir CSV dosyasına kaydedin (isteğe bağlı)
books_df.to_csv('books_data.csv', index=False)

In [ ]:
ads = pd.read_csv('books_data.csv', index_col=['sales'], parse_dates=['sales'])

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(ads.family)
plt.title('Ads watched (hourly data)')
plt.grid(True)
plt.show()

In [ ]:
# 'family' sütununda sadece 'BOOKS' ve belirli bir 'store_nbr' değerini içeren satırları seçin
store_number = 11  # İstediğiniz mağaza numarasını buraya girin
books_store_df = df[(df['family'] == 'GROCERY I') & (df['store_nbr'] == store_number)]

# Tarih sütununu datetime formatına çevirin
books_store_df['date'] = pd.to_datetime(books_store_df['date'])

# Tarih sütununu index olarak ayarlayın
books_store_df.set_index('date', inplace=True)

# Zaman serisi grafiği çizin
plt.figure(figsize=(15, 7))
plt.plot(books_store_df['sales'], label='Sales')
plt.title('Sales of Grocery I Over Time')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Veriyi haftalık olarak yeniden örnekleyin ve satışların toplamını hesaplayın
weekly_sales = books_store_df['sales'].resample('W').sum()

# Zaman serisi grafiği çizin
plt.figure(figsize=(15, 7))
plt.plot(weekly_sales, label='Weekly Sales')
plt.title('Weekly GROCERY I of Books Over Time')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Import
train = pd.read_csv("../input/store-sales-time-series-forecasting/train.csv")
test = pd.read_csv("../input/store-sales-time-series-forecasting/test.csv")
stores = pd.read_csv("../input/store-sales-time-series-forecasting/stores.csv")
#sub = pd.read_csv("../input/store-sales-time-series-forecasting/sample_submission.csv")   
transactions = pd.read_csv("../input/store-sales-time-series-forecasting/transactions.csv").sort_values(["store_nbr", "date"])


# Datetime
train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)
transactions["date"] = pd.to_datetime(transactions.date)

# Data types
train.onpromotion = train.onpromotion.astype("float16")
train.sales = train.sales.astype("float32")
stores.cluster = stores.cluster.astype("int8")

train.head()

# Veriyi yükleyin
df = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')

# 'family' sütununda sadece 'BOOKS' ve belirli bir 'store_nbr' değerini içeren satırları seçin
store_number = 11  # İstediğiniz mağaza numarasını buraya girin
books_store_df = df[(df['family'] == 'GROCERY I') & (df['store_nbr'] == store_number)]

# Tarih sütununu datetime formatına çevirin
books_store_df['date'] = pd.to_datetime(books_store_df['date'])

# Tarih sütununu index olarak ayarlayın
books_store_df.set_index('date', inplace=True)

# Zaman serisi grafiği çizin
plt.figure(figsize=(15, 7))
plt.plot(books_store_df['sales'], label='Sales')
plt.title('Sales of Grocery I Over Time')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.show()

# 'date' sütununda sadece '2013-01-01' değerini içeren satırları seçin
date_df = df[(df['date'] == '2013-01-01')]


# Seçilen veriyi kontrol edin,
print(date_df.head())

In [ ]:
# Ortalama ve standart sapmayı hesaplayın
mean_sales = weekly_sales.mean()
std_sales = weekly_sales.std()

# Aykırı değerlerin belirlenmesi
outliers = weekly_sales[(weekly_sales < mean_sales - 3 * std_sales) | (weekly_sales > mean_sales + 3 * std_sales)]

# Zaman serisi grafiği ve aykırı değerleri çizin
plt.figure(figsize=(15, 7))
plt.plot(weekly_sales, label='Weekly Sales', color='blue')
plt.scatter(outliers.index, outliers, color='red', label='Outliers')
plt.title('Weekly Sales of Books Over Time with Outliers')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 'family' sütununda sadece 'GROCERY I' ve belirli bir 'store_nbr' değerini içeren satırları seçin
store_number = 11  # İstediğiniz mağaza numarasını buraya girin
grocery_store_df = df[(df['family'] == 'GROCERY I') & (df['store_nbr'] == store_number)]

# Tarih sütununu datetime formatına çevirin
grocery_store_df['date'] = pd.to_datetime(grocery_store_df['date'])

# Tarih sütununu index olarak ayarlayın
grocery_store_df.set_index('date', inplace=True)

# Ortalama ve standart sapmayı hesaplayın
mean_sales = grocery_store_df['sales'].mean()
std_sales = grocery_store_df['sales'].std()

# Aykırı değerlerin belirlenmesi
outliers = grocery_store_df[(grocery_store_df['sales'] < mean_sales - 3 * std_sales) | (grocery_store_df['sales'] > mean_sales + 3 * std_sales)]

# Zaman serisi grafiği ve aykırı değerleri çizin
plt.figure(figsize=(15, 7))
plt.plot(grocery_store_df['sales'], label='Daily Sales', color='blue')
plt.scatter(outliers.index, outliers['sales'], color='red', label='Outliers')
plt.title('Daily Sales of Grocery I Over Time with Outliers')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
 #Importing everything from above

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# Hareketli ortalama hesaplama fonksiyonu
def moving_average(series, window):
    return series.rolling(window=window).mean()

# Hareketli ortalama değerini hesaplayın (örn. 7 günlük)
window_size = 7
grocery_store_df['moving_average'] = moving_average(grocery_store_df['sales'], window_size)
#moving_average(grocery_store_df, window_size)
# Grafiği oluşturun

plt.figure(figsize=(12, 6))
plt.plot(grocery_store_df['sales'], label='Daily Sales')
plt.plot(grocery_store_df['moving_average'], label='7-Day Moving Average', color='orange')
plt.title('Daily Sales and 7-Day Moving Average for Grocery I')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(weekly_sales, label='Weekly Sales', color='blue')
plt.plot(grocery_store_df['moving_average'], label='7-Day Moving Average', color='orange')
plt.title('Weekly Sales of Books Over Time with Outliers')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 'family' sütununda sadece 'GROCERY I' ve belirli bir 'store_nbr' değerini içeren satırları seçin
store_number = 1  # İstediğiniz mağaza numarasını buraya girin
grocery_store_df = df[(df['family'] == 'GROCERY I') & (df['store_nbr'] == store_number)]

# Tarih sütununu datetime formatına çevirin
grocery_store_df['date'] = pd.to_datetime(grocery_store_df['date'])

# Tarih sütununu index olarak ayarlayın
grocery_store_df.set_index('date', inplace=True)

# Hareketli ortalama ve anomali tespiti fonksiyonu
def plotMovingAverage(series, window, plot_intervals=False, scale=1.96, plot_anomalies=False):
    rolling_mean = series.rolling(window=window).mean()

    plt.figure(figsize=(15, 5))
    plt.title("Moving average\n window size = {}".format(window))
    plt.plot(rolling_mean, "g", label="Rolling mean trend")

    # Plot confidence intervals for smoothed values
    if plot_intervals:
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bond = rolling_mean - (mae + scale * deviation)
        upper_bond = rolling_mean + (mae + scale * deviation)
        plt.plot(upper_bond, "r--", label="Upper Bond / Lower Bond")
        plt.plot(lower_bond, "r--")

        # Having the intervals, find abnormal values
        if plot_anomalies:
            anomalies = pd.DataFrame(index=series.index)
            anomalies[series < lower_bond] = series[series < lower_bond]
            anomalies[series > upper_bond] = series[series > upper_bond]
            plt.plot(anomalies, "ro", markersize=5)
        
    plt.plot(series, label="Actual values")
    plt.legend(loc="upper left")
    plt.grid(True)
    plt.show()

# Satış verileri serisi
sales_series = grocery_store_df['sales']

# Hareketli ortalama ve anomali tespitini içeren grafiği oluşturun
# 'family' sütununda sadece 'GROCERY I' ve belirli bir 'store_nbr' değerini içeren satırları seçin
store_number = 1  # İstediğiniz mağaza numarasını buraya girin
grocery_store_df = df[(df['family'] == 'GROCERY I') & (df['store_nbr'] == store_number)]

# Tarih sütununu datetime formatına çevirin
grocery_store_df['date'] = pd.to_datetime(grocery_store_df['date'])

# Tarih sütununu index olarak ayarlayın
grocery_store_df.set_index('date', inplace=True)

# Hareketli ortalama ve anomali tespiti fonksiyonu
def plotMovingAverage(series, window, plot_intervals=False, scale=1.96, plot_anomalies=False):
    rolling_mean = series.rolling(window=window).mean()

    plt.figure(figsize=(15, 5))
    plt.title("Moving average\n window size = {}".format(window))
    plt.plot(rolling_mean, "g", label="Rolling mean trend")

    # Plot confidence intervals for smoothed values
    if plot_intervals:
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bond = rolling_mean - (mae + scale * deviation)
        upper_bond = rolling_mean + (mae + scale * deviation)
        plt.plot(upper_bond, "r--", label="Upper Bond / Lower Bond")
        plt.plot(lower_bond, "r--")

        # Having the intervals, find abnormal values
        if plot_anomalies:
            anomalies = pd.DataFrame(index=series.index)
            anomalies[series < lower_bond] = series[series < lower_bond]
            anomalies[series > upper_bond] = series[series > upper_bond]
            plt.plot(anomalies, "ro", markersize=5)
        
    plt.plot(series, label="Actual values")
    plt.legend(loc="upper left")
    plt.grid(True)
    plt.show()

# Satış verileri serisi
sales_series = grocery_store_df['sales']

# Hareketli ortalama ve anomali tespitini içeren grafiği oluşturun
plotMovingAverage(sales_series, window=7, plot_intervals=True, plot_anomalies=True)

In [ ]:
plotMovingAverage(sales_series, 7)

In [ ]:
import pandas as pd
import plotly.express as px

# Adım 1: Verilerin Gruplanması ve Birleştirilmesi
# Verilerinizi burada uygun şekilde değiştirin
train_grouped = train.groupby(["date", "store_nbr"]).sales.sum().reset_index()
temp = pd.merge(train_grouped, transactions, how="left", on=["date", "store_nbr"])

# Adım 2: Spearman Korelasyonunu Hesaplama ve Yazdırma
spearman_corr = temp.corr(method="spearman").loc["transactions", "sales"]
print(f"Spearman Correlation between Total Sales and Transactions: {spearman_corr:.4f}")

# Adım 3: Çizgi Grafiğinin Oluşturulması
# Verilerinizi burada uygun şekilde değiştirin
sorted_transactions = transactions.sort_values(["store_nbr", "date"])
fig = px.line(
    sorted_transactions,
    x="date",
    y="transactions",
    color="store_nbr",
    title="Transactions"
)
fig.show()
